# Text Generation using LSTM (Word-Level)

This mini-project demonstrates how to build a simple **word-level text generation model** using **LSTM (Long Short-Term Memory)** neural networks.

We use a small dummy dataset consisting of simple English sentences. The goal is to train an LSTM model to predict the next word in a sequence based on the previous words.

### Project Goals
- Tokenize and preprocess text data at the word level
- Prepare training sequences suitable for LSTM
- Build and train an LSTM-based language model
- Generate new text given a starting seed
- Understand key NLP concepts like:
  - Word tokenization
  - Sequence generation
  - Padding
  - LSTM and dense layers

### Tools & Libraries
- Python
- TensorFlow / Keras
- NumPy


In [57]:
import nltk
nltk.download("gutenberg")
nltk.download("punkt")

[nltk_data] Downloading package gutenberg to
[nltk_data]     /home/omkarjadhav/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/omkarjadhav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Step 1 – Dataset Preparation

##### Load the text

In [78]:
from nltk.corpus import gutenberg


# Dummy text corpus
sample_text = """Once upon a time in a quiet village, there lived a curious girl named Lily. 
Every morning, she wandered through the fields, chasing butterflies and collecting colorful stones. 
She believed every stone had a story, a secret it whispered only to her. 
One day, she found a glowing pebble near the old oak tree. 
From that moment, strange dreams visited her, showing her distant lands and talking animals. 
Lily knew it was the beginning of an adventure. She packed her bag, hugged her family, and stepped into the forest, following the silent song only she could hear."""


### Step 2: Text Cleaning and Tokenization

In [79]:
import re
from nltk.tokenize import word_tokenize

# Lowercase
text = sample_text.lower()

# # Remove unwanted characters (keep letters and basic punctuation)
text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

tokens = word_tokenize(text)
print("Total Tokens: ", len(tokens))
print("First 50 Tokens: ", tokens[:50])

Total Tokens:  97
First 50 Tokens:  ['once', 'upon', 'a', 'time', 'in', 'a', 'quiet', 'village', 'there', 'lived', 'a', 'curious', 'girl', 'named', 'lily', 'every', 'morning', 'she', 'wandered', 'through', 'the', 'fields', 'chasing', 'butterflies', 'and', 'collecting', 'colorful', 'stones', 'she', 'believed', 'every', 'stone', 'had', 'a', 'story', 'a', 'secret', 'it', 'whispered', 'only', 'to', 'her', 'one', 'day', 'she', 'found', 'a', 'glowing', 'pebble', 'near']


### Step 3: Sequence Creation for Training

Now that we have a cleaned list of words `(tokens)`, we'll prepare training sequences.

Goal:
* We want to create many input sequences where each sequence is a few words long, and the next word is the **label**.


For example:

Input Sequence: ["alice", "was", "beginning", "to"]    

Label: "get"


In [80]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [81]:
# Initialize the tokenizer and fit on text

"""Below step builds the vocabulary from the list of tokens (words). 
   Each unique word gets assigned an integer index. 
   We're using those integer indexes as sequences values. Ex: "alices" word have index 298 which we're using to represent that word.
"""
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokens)      # Learn the vocabulary from the text

# Convert words to integers
sequences = tokenizer.texts_to_sequences([tokens])[0]
print("sequences size: ", len(sequences))
print("Sequence: ", sequences[:10])


# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print("\nVocabulary Size: ", vocab_size)


# Create input sequences and labels
input_sequences = []
for i in range(1, len(sequences)):
    input_sequences.append(sequences[:i+1])
print("\nfirst 5 input sequences:\n", input_sequences[:5])



# Pad sequences to same length
max_seq_len = len(input_sequences[-1])
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre')
print("\nfirst 5 input sequences After padding:\n", input_sequences[:5])

# Split into input (X) and label (y)
input_sequences = np.array(input_sequences)
"""Below code is used to get values from 2D array"""
x = input_sequences[:, :-1]  # All rows and all columns expect last one
y = input_sequences[:, -1]   # All rows and last column

print("\nInput features type: ", type(x))
print("Label type: ", type(y))

y = to_categorical(y, num_classes=vocab_size)

sequences size:  97
Sequence:  [10, 11, 1, 12, 13, 1, 14, 15, 16, 17]

Vocabulary Size:  75

first 5 input sequences:
 [[10, 11], [10, 11, 1], [10, 11, 1, 12], [10, 11, 1, 12, 13], [10, 11, 1, 12, 13, 1]]

first 5 input sequences After padding:
 [[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10
  11]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10 11
   1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  

#### Step 4: Building the LSTM Text Generation Model

In [82]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=max_seq_len - 1))
model.add(LSTM(100))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 96, 64)            4800      
                                                                 
 lstm_7 (LSTM)               (None, 100)               66000     
                                                                 
 dense_7 (Dense)             (None, 75)                7575      
                                                                 
Total params: 78,375
Trainable params: 78,375
Non-trainable params: 0
_________________________________________________________________


2025-06-01 10:51:28.632938: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-06-01 10:51:28.633830: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-06-01 10:51:28.634908: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [83]:
history = model.fit(x, y, epochs=100, verbose=1)

Epoch 1/100


2025-06-01 10:51:29.125466: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-06-01 10:51:29.127415: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-06-01 10:51:29.128810: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

3/3 [==============================] - 2s 105ms/step - loss: 4.3182 - accuracy: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 0s 93ms/step - loss: 4.3089 - accuracy: 0.0729
Epoch 3/100
3/3 [==============================] - 0s 95ms/step - loss: 4.3003 - accuracy: 0.1146
Epoch 4/100
3/3 [==============================] - 0s 80ms/step - loss: 4.2900 - accuracy: 0.1354
Epoch 5/100
3/3 [==============================] - 0s 102ms/step - loss: 4.2768 - accuracy: 0.1146
Epoch 6/100
3/3 [==============================] - 0s 86ms/step - loss: 4.2476 - accuracy: 0.0625
Epoch 7/100
3/3 [==============================] - 0s 86ms/step - loss: 4.1845 - accuracy: 0.0625
Epoch 8/100
3/3 [==============================] - 0s 89ms/step - loss: 4.1352 - accuracy: 0.0625
Epoch 9/100
3/3 [==============================] - 0s 80ms/step - loss: 4.1022 - accuracy: 0.0938
Epoch 10/100
3/3 [==============================] - 0s 83ms/step - loss: 4.0708 - accuracy: 0.0729
Epoch 11/100
3/3 [=======

In [84]:
def generate_text(seed_text, next_words, model, max_sequence_len, tokenizer):
    for _ in range(next_words):
        # print(tokenizer.texts_to_sequences([seed_text]))
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        # print("predicted_probs: ", predicted_probs)
        predicted_index = np.argmax(predicted_probs, axis=-1)[0]

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break

        seed_text += " " + output_word
    return seed_text


In [86]:
seed_text = "Once upon a"
next_words = 10
generated = generate_text(seed_text, next_words, model, max_seq_len, tokenizer)
print("Generated text:\n", generated)

Generated text:
 Once upon a a time a a village there lived a curious girl


In [87]:
# Save model
model.save("text_gen_lstm_model.h5")

# Save tokenizer
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


In [88]:
from tensorflow.keras.models import load_model

model = load_model("text_gen_lstm_model.h5")

with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)


2025-06-01 10:52:01.868319: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-06-01 10:52:01.869487: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-06-01 10:52:01.870447: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus